In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

from sklearn.metrics import accuracy_score, confusion_matrix, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

factor_2017_18.csv
gamescores_2017_18.csv
gamescores_2017_18_full.csv
gender_submission.csv
test.csv
train.csv



In [3]:
data = pd.read_csv("../data/train.csv")

In [4]:
data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [5]:
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
data.drop(['PassengerId','Ticket'], axis = 1, inplace = True)

In [7]:
data['title'] = data['Name'].apply(lambda x: x.split(', ')[1].split('. ')[0])
data['last_name'] = data['Name'].apply(lambda x: x.split(', ')[0])

In [8]:
# infer age from name title by group average
kf = StratifiedKFold(5, shuffle=False, random_state= 233)

In [9]:
data['age_title_enc'] = np.nan

for tr_ind, val_ind in kf.split(data, data.Survived):
    X_tr, X_val = data.iloc[tr_ind], data.iloc[val_ind]
    X_val['age_title_enc'] = X_val['title'].map(X_tr.groupby('title')['Age'].mean())
    data.iloc[val_ind] = X_val

In [10]:
prior = data['Age'].mean()
data['age_title_enc'].fillna(prior, inplace=True)

In [11]:
data['Age'].fillna(data['age_title_enc'], inplace = True)
data.drop('age_title_enc', axis = 1, inplace = True)

In [12]:
data[data['Cabin'].str.contains(pat = '[A-Z] [A-Z][0-9]*').fillna(False)]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,title,last_name
75,0,3,"Moen, Mr. Sigurd Hansen",male,25.000000,0,0,7.6500,F G73,S,Mr,Moen
128,1,3,"Peter, Miss. Anna",female,22.566372,1,1,22.3583,F E69,C,Miss,Peter
699,0,3,"Humblen, Mr. Adolf Mathias Nicolai Olsen",male,42.000000,0,0,7.6500,F G63,S,Mr,Humblen
715,0,3,"Soholt, Mr. Peter Andreas Lauritz Andersen",male,19.000000,0,0,7.6500,F G73,S,Mr,Soholt


In [13]:
data[data['Cabin'].str.contains(pat = 'T').fillna(False)]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,title,last_name
339,0,1,"Blackwell, Mr. Stephen Weart",male,45.0,0,0,35.5,T,S,Mr,Blackwell


In [28]:
data[data['Cabin'].str.contains(pat = '[A-Z][0-9]* ').fillna(False)].head(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,title,last_name,cabin_count,cabin_letter,cabin_number
27,0,1,"Fortune, Mr. Charles Alexander",male,19.000000,3,2,263.0000,C23 C25 C27,S,Mr,Fortune,3,C,23 C25 C27
75,0,3,"Moen, Mr. Sigurd Hansen",male,25.000000,0,0,7.6500,F G73,S,Mr,Moen,2,F,G73
88,1,1,"Fortune, Miss. Mabel Helen",female,23.000000,3,2,263.0000,C23 C25 C27,S,Miss,Fortune,3,C,23 C25 C27
97,1,1,"Greenfield, Mr. William Bertram",male,23.000000,0,1,63.3583,D10 D12,C,Mr,Greenfield,2,D,10 D12
118,0,1,"Baxter, Mr. Quigg Edmond",male,24.000000,0,1,247.5208,B58 B60,C,Mr,Baxter,2,B,58 B60
128,1,3,"Peter, Miss. Anna",female,22.566372,1,1,22.3583,F E69,C,Miss,Peter,2,F,E69
297,0,1,"Allison, Miss. Helen Loraine",female,2.000000,1,2,151.5500,C22 C26,S,Miss,Allison,2,C,22 C26
299,1,1,"Baxter, Mrs. James (Helene DeLaudeniere Chaput)",female,50.000000,0,1,247.5208,B58 B60,C,Mrs,Baxter,2,B,58 B60
305,1,1,"Allison, Master. Hudson Trevor",male,0.920000,1,2,151.5500,C22 C26,S,Master,Allison,2,C,22 C26
311,1,1,"Ryerson, Miss. Emily Borie",female,18.000000,2,2,262.3750,B57 B59 B63 B66,C,Miss,Ryerson,4,B,57 B59 B63 B66


In [16]:
data['Cabin'].filter(regex = 'C')

Series([], Name: Cabin, dtype: object)

In [17]:
data['cabin_count'] = data['Cabin'].apply(lambda x: len(str(x).split(' '))) # assuming those missing has one cabin
data['cabin_letter'] = data['Cabin'].apply(lambda x: 'NA' if pd.isnull(x) else set([i[0] for i in x.split(' ')]))

In [27]:
data[data['cabin_count'] > 1].head(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,title,last_name,cabin_count,cabin_letter,cabin_number
27,0,1,"Fortune, Mr. Charles Alexander",male,19.000000,3,2,263.0000,C23 C25 C27,S,Mr,Fortune,3,C,23 C25 C27
75,0,3,"Moen, Mr. Sigurd Hansen",male,25.000000,0,0,7.6500,F G73,S,Mr,Moen,2,F,G73
88,1,1,"Fortune, Miss. Mabel Helen",female,23.000000,3,2,263.0000,C23 C25 C27,S,Miss,Fortune,3,C,23 C25 C27
97,1,1,"Greenfield, Mr. William Bertram",male,23.000000,0,1,63.3583,D10 D12,C,Mr,Greenfield,2,D,10 D12
118,0,1,"Baxter, Mr. Quigg Edmond",male,24.000000,0,1,247.5208,B58 B60,C,Mr,Baxter,2,B,58 B60
128,1,3,"Peter, Miss. Anna",female,22.566372,1,1,22.3583,F E69,C,Miss,Peter,2,F,E69
297,0,1,"Allison, Miss. Helen Loraine",female,2.000000,1,2,151.5500,C22 C26,S,Miss,Allison,2,C,22 C26
299,1,1,"Baxter, Mrs. James (Helene DeLaudeniere Chaput)",female,50.000000,0,1,247.5208,B58 B60,C,Mrs,Baxter,2,B,58 B60
305,1,1,"Allison, Master. Hudson Trevor",male,0.920000,1,2,151.5500,C22 C26,S,Master,Allison,2,C,22 C26
311,1,1,"Ryerson, Miss. Emily Borie",female,18.000000,2,2,262.3750,B57 B59 B63 B66,C,Miss,Ryerson,4,B,57 B59 B63 B66


In [19]:
data['cabin_letter'] = data['Cabin'].str[0]
data['cabin_number'] = data['Cabin'].apply(lambda x:  np.nan if pd.isnull(x) else -999 if len(x) == 1 else x[1:])

In [20]:
data.cabin_letter.value_counts()

C    59
B    47
D    33
E    32
A    15
F    13
G     4
T     1
Name: cabin_letter, dtype: int64

In [26]:
data['cabin_number'].value_counts()[:10]

33            7
6             6
2             5
101           5
49            5
20            5
50            4
96 B98        4
23 C25 C27    4
-999          4
Name: cabin_number, dtype: int64

In [25]:
data.groupby('Cabin')['Fare'].agg(['mean','std']).head(10)

,mean,std
Cabin,,
A10,40.1250,NaN
A14,52.0000,NaN
A16,39.6000,NaN
A19,26.0000,NaN
A20,56.9292,NaN
A23,30.0000,NaN
A24,50.4958,NaN
A26,35.5000,NaN
A31,31.0000,NaN


In [30]:
data = pd.get_dummies(data.drop(['Name', 'Cabin'], axis = 1))

X = data.drop('Survived',axis = 1)
y = data['Survived']

In [43]:
si = Imputer(strategy = 'median', axis = 1)
si.fit(X)

/home/fzhang/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


Imputer(axis=1, copy=True, missing_values='NaN', strategy='median', verbose=0)

In [44]:

X_i = si.transform(X)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(
    X_i, y, test_size = .5, random_state = 233
)

In [49]:
xlf = XGBClassifier()
xlf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [50]:
xlf_prob = xlf.predict_proba(X_test)[:,1]
xlf_pred = xlf.predict(X_test)

In [51]:

print(confusion_matrix(xlf_pred, y_test))
print(accuracy_score(xlf_pred, y_test))

[[228  46]
 [ 41 131]]
0.804932735426009


In [52]:
importance = pd.DataFrame()
importance['Feature'] = X.columns
importance['importance'] = xlf.feature_importances_

In [38]:
importance.sort_values(ascending = False, by = 'importance').head(10)

,Feature,importance
4,Fare,0.375874
1,Age,0.234266
0,Pclass,0.055944
2,SibSp,0.055944
6,Sex_female,0.045455
3,Parch,0.038462
8,Embarked_C,0.036713
699,cabin_letter_E,0.034965
23,title_Mrs,0.029720
22,title_Mr,0.029720
